# Convergence Analysis of Adapted Empirical Measure and Discrete Optimal Transport Performance

## Overview
This notebook investigates the convergence properties of the adapted empirical measure and evaluates the performance of discrete optimal transport calculations on larger trees. The study consists of:

1. **Generating and Comparing Two Brownian Motion Adapted Trees**
   - We sample paths from two Brownian motions.
   - Construct adapted trees for each.
   - Compare the values obtained against benchmark values.

2. **Benchmarking**
   - Compute the theoretical adapted Wasserstein distance.
   - Validate results against numerical computations using discrete trees.

This approach helps assess the accuracy and efficiency of the empirical adapted measure in capturing the underlying stochastic process.

In [13]:
import numpy as np
import os
import sys
import warnings
import time

# Define paths to relevant modules
measure_sampling_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Measure_sampling')
trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Trees')
Benchmark_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/Benchmark_value_Gausian')
awd_trees_path = os.path.abspath('/Users/rubenbontorno/Documents/Master_Thesis/Code/AWD_numerics/AWD_trees')

# Add paths to sys.path
for path in [measure_sampling_path, trees_path, Benchmark_path, awd_trees_path]:
    if path not in sys.path:
        sys.path.append(path)

# Import necessary modules
from Gen_Path_and_AdaptedTrees import generate_adapted_tree
from Tree_Node import *
from TreeAnalysis import *
from TreeVisualization import *
from Save_Load_trees import *
from Tree_AWD_utilities import *
from Comp_AWD2_Gaussian import build_mean_and_cov, adapted_wasserstein_squared
from Gurobi_AOT import *
from Nested_Dist_Algo import compute_nested_distance

# Suppress sklearn warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [17]:
# Set parameters
num_paths = 5000   # Number of sample paths
time_steps = 4   # Number of time steps per path
scale1 = 1
scale2 = 0.5
use_weights = 1

# Generate adapted trees for Brownian motion
print("Generating adapted tree from the first Brownian motion...")
bm1_root, bm1_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale1, use_weights=use_weights, model='brownian', return_times=True)

print("Generating adapted tree from the second Brownian motion...")
bm2_root, bm2_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale2, use_weights=use_weights, model='brownian', return_times=True)

print(f"Brownian Motion Adapted Tree 1 generated successfully in {sum(bm1_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm1_time_tdetail[0]:.2f}s, Measure k-means: {bm1_time_tdetail[1]:.2f}s, Tree construction: {bm1_time_tdetail[2]:.2f}s")

print(f"Brownian Motion Adapted Tree 2 generated successfully in {sum(bm2_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm2_time_tdetail[0]:.2f}s, Measure k-means: {bm2_time_tdetail[1]:.2f}s, Tree construction: {bm2_time_tdetail[2]:.2f}s")

Generating adapted tree from the first Brownian motion...
Generating adapted tree from the second Brownian motion...
Brownian Motion Adapted Tree 1 generated successfully in 78.64 seconds.
Time breakdown: Sample generation: 0.03s, Measure k-means: 78.57s, Tree construction: 0.04s
Brownian Motion Adapted Tree 2 generated successfully in 61.54 seconds.
Time breakdown: Sample generation: 0.03s, Measure k-means: 60.75s, Tree construction: 0.76s


In [18]:
# Benchmarking Adapted Wasserstein Distance

# Define parameters
a, b = 0, 0
var_a, var_b = scale1**2, scale2**2
t = time_steps-1

# Build mean and covariance matrices for both processes
a_vec, A_mat = build_mean_and_cov(t, mean_val=a, var_factor=var_a)
b_vec, B_mat = build_mean_and_cov(t, mean_val=b, var_factor=var_b)

# Compute adapted Wasserstein squared distance
distance_squared = adapted_wasserstein_squared(a_vec, A_mat, b_vec, B_mat)
distance = np.sqrt(distance_squared)

print(f"Adapted Wasserstein Squared Distance: {distance_squared:.4f}")
print(f"Adapted Wasserstein Distance: {distance:.4f}")

Adapted Wasserstein Squared Distance: 1.5000
Adapted Wasserstein Distance: 1.2247


In [19]:
# Compute optimal transport using POT solver
print("\nComputing adapted optimal transport using POT solver...")
max_depth = get_depth(bm1_root)
start_time = time.time()
distance_pot = compute_nested_distance(
    bm1_root, bm2_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0, power = 2
)
elapsed_time_pot = time.time() - start_time
print(f"POT Solver Distance: {distance_pot:.4f}, Computation Time: {elapsed_time_pot:.2f}s")


Computing adapted optimal transport using POT solver...
POT Solver Distance: 1.8265, Computation Time: 1392.99s


In [ ]:
# Compute Optimal Transport Distances between Trees
formatted_tree_1 = get_sample_paths(bm1_root)
formatted_tree_2 = get_sample_paths(bm2_root)

def cost_function(x, y):
    """Cost function: L1 distance."""
    return np.sum(np.abs(x - y)**2)

# Compute optimal transport using Gurobi
print("\nComputing adapted optimal transport using Gurobi...")
start_time = time.time()
val_gurobi = gurobi_bm(
    [formatted_tree_1, formatted_tree_2],
    f=cost_function,
    r_opti=0,
    causal=1,
    anticausal=1,
    outputflag=0
)
elapsed_time_gurobi = time.time() - start_time
print(f"Gurobi Optimal Transport Value: {val_gurobi:.4f}, Computation Time: {elapsed_time_gurobi:.2f}s")

In [ ]:
# Compute optimal transport using linear programming
print("\nComputing adapted optimal transport using LP solver...")
max_depth = get_depth(bm1_root)
start_time = time.time()
distance_lp = compute_nested_distance(
    bm1_root, bm2_root, max_depth, method="solver_lp", return_matrix=False, lambda_reg=0, power =2
)
elapsed_time_lp = time.time() - start_time
print(f"LP Solver Distance: {distance_lp:.4f}, Computation Time: {elapsed_time_lp:.2f}s")

In [ ]:
# Compute Sinkhorn distance with regularization
print("\nComputing adapted optimal transport using Sinkhorn regularization...")
max_depth = get_depth(bm1_root)
start_time = time.time()
distance_sinkhorn = compute_nested_distance(
    bm1_root, bm2_root, max_depth, method="Sinkhorn", return_matrix=False, lambda_reg=8, power = 2
)
elapsed_time_sinkhorn = time.time() - start_time
print(f"Sinkhorn Regularized Distance: {distance_sinkhorn:.4f}, Computation Time: {elapsed_time_sinkhorn:.2f}s")

In [ ]:
# Generate adapted trees for Brownian motion and financial model
print("Generating adapted tree from Brownian motion...")
bm_root, bm_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale, use_weights=use_weights, model='brownian', return_times=True)

print("Generating adapted tree from Financial Model...")
fin_root, fin_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=scale, use_weights=use_weights, model='financial', return_times=True)

print(f"Brownian Motion Adapted Tree generated successfully in {sum(bm_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm_time_tdetail[0]:.2f}s, Measure k-means: {bm_time_tdetail[1]:.2f}s, Tree construction: {bm_time_tdetail[2]:.2f}s")

print(f"Financial Model Adapted Tree generated successfully in {sum(fin_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {fin_time_tdetail[0]:.2f}s, Measure k-means: {fin_time_tdetail[1]:.2f}s, Tree construction: {fin_time_tdetail[2]:.2f}s")



# Set parameters
num_paths = 200   # number of sample paths
time_steps = 5   # number of time steps per path
scale = 1
use_weights = 1


# Generate adapted trees for Brownian motion and financial model
print("Generating adapted tree from Brownian motion...")
bm_root, bm_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=1, use_weights=use_weights, model='brownian', return_times=True)

print("Generating adapted tree from Financial Model...")
fin_root, fin_time_tdetail = generate_adapted_tree(num_paths=num_paths, time_steps=time_steps, scale=0.5**2, use_weights=use_weights, model='brownian', return_times=True)

print(f"Brownian Motion Adapted Tree generated successfully in {sum(bm_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {bm_time_tdetail[0]:.2f}s, Measure k-means: {bm_time_tdetail[1]:.2f}s, Tree construction: {bm_time_tdetail[2]:.2f}s")

print(f"Financial Model Adapted Tree generated successfully in {sum(fin_time_tdetail):.2f} seconds.")
print(f"Time breakdown: Sample generation: {fin_time_tdetail[0]:.2f}s, Measure k-means: {fin_time_tdetail[1]:.2f}s, Tree construction: {fin_time_tdetail[2]:.2f}s")

In [ ]:
start_time = time.time()

max_depth = get_depth(bm_root)
distance = compute_nested_distance(
    bm_root, fin_root, max_depth, method="solver_pot", return_matrix=False, lambda_reg=0
)

end_time = time.time()
elapsed_time = end_time - start_time

print(distance)
print(elapsed_time)